In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import sys
sys.path.append("../")
import torch
assert torch.cuda.is_available()
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import torch.nn as nn
from tqdm.auto import tqdm
import json
from train import eval
from Utils import load

In [2]:
pruners = np.array(
    [
        # 'cs',
        # 'grasp',
        # 'sf',
        # 'mica_cs_shuffle',
        # 'mica_grasp_shuffle',
        # 'mica_sf_shuffle',
        # 'mica_erk_shuffle',
        'mica_igq_shuffle',
        # 'mica_cs_rand',
        # 'mica_grasp_rand',
        # 'mica_sf_rand',
        # 'mica_erk_rand',
        # 'mica_igq_rand'
        # 'mica_igq_shuffle',
        
    ],
    dtype=object
    )

colors = {
    'cs'         : plt.cm.tab10(9),
    'grasp'      : plt.cm.tab10(8),
    'sf'         : plt.cm.tab10(6),
    'mica_cs'    : plt.cm.tab10(9),
    'mica_grasp' : plt.cm.tab10(8),
    'mica_sf'    : plt.cm.tab10(6),
    'mica_erk'   : plt.cm.tab10(4),
    'mica_igq'   : plt.cm.tab10(3),
}

markers = {
    'cs'         : "^", 
    'grasp'      : "h", 
    'sf'         : "H", 
    'mica_cs'    : "^", 
    'mica_grasp' : "h", 
    'mica_sf'    : "H",
    'mica_erk'   : "p", 
    'mica_igq'   : "*"
}

pruner_names = {
    'cs'                 : 'SNIP (Original)',
    'grasp'              : 'GraSP (Original)',
    'sf'                 : 'SynFlow (Original)',
    'mica_cs_shuffle'    : 'SNIP (Random)',
    'mica_grasp_shuffle' : 'GraSP (Random)',
    'mica_sf_shuffle'    : 'SynFlow (Random)',
    'mica_erk_shuffle'   : 'ERK (Random)',
    'mica_igq_shuffle'   : 'IGQ (Random)',
    'mica_cs_rand'       : '$\\bf{MiCA}$-$\\bf{SNIP (Rand)}$',
    'mica_grasp_rand'    : '$\\bf{MiCA}$-$\\bf{GraSP (Rand)}$',
    'mica_sf_rand'       : '$\\bf{MiCA}$-$\\bf{SynFlow (Rand)}$',
    'mica_erk_rand'      : '$\\bf{MiCA}$-$\\bf{ERK (Rand)}$',
    'mica_igq_rand'      : '$\\bf{MiCA}$-$\\bf{IGQ (Rand)}$',
    }

pepochs = {
    'cs'         : '1',
    'grasp'      : '1',
    'sf'         : '100',
    'mica_cs'    : '1',
    'mica_grasp' : '1',
    'mica_sf'    : '1',
    'mica_erk'   : '1',
    'mica_igq'   : '1',
}

for pruner in pruners:
    if '_shuffle' in pruner:
        colors[pruner]  = colors[pruner.replace('_shuffle', '')]
        markers[pruner]  = markers[pruner.replace('_shuffle', '')]
        pepochs[pruner] = pepochs[pruner.replace('_shuffle', '')]
    elif '_rand' in pruner:
        colors[pruner]  = colors[pruner.replace('_rand', '')]
        markers[pruner]  = markers[pruner.replace('_rand', '')]
        pepochs[pruner] = pepochs[pruner.replace('_rand', '')]

ratios = [
    # '0.5', '1.0', '1.5', '2.0', 
    '2.5', 
    # '3.0', '3.5', '4.0', 
    ]
expid = 'cifar10_lottery_resnet20'
runs = [0, 1, 2]

max_run_num = len(runs)

print(pruners)

['mica_igq_shuffle']


In [3]:
def get_experimentid(pruner, ratio, pepochs, run_num):
    if '_shuffle' in pruner:
        base_dir = f'../Results/data/singleshot/{expid}_shuffle'
        p_name = pruner.replace('_shuffle', '')
    elif '_min' in pruner:
        base_dir = f'../Results/data/singleshot/{expid}_min'
        p_name = pruner.replace('_min', '')
    elif '_max' in pruner:
        base_dir = f'../Results/data/singleshot/{expid}_max'
        p_name = pruner.replace('_max', '')
    elif '_rand' in pruner:
        base_dir = f'../Results/data/singleshot/{expid}_rand'
        p_name = pruner.replace('_rand', '')
    else:
        base_dir = f'../Results/data/singleshot/{expid}'
        p_name = pruner
    b_dir  = f'{base_dir}'
    experimentid =  f'{b_dir}/' \
                    f'{p_name}-{ratio}-{pepochs}/' \
                    f'run_{run_num}/' 
    return experimentid

def zero_to_nan(values):
    return [float('nan') if x==0 else x for x in values]

def get_ordered_list(l, order):
    out = []
    for i in order:
        out.append(l[i])
    return out

In [ ]:
expid = 'cifar100_lottery_resnet20'
ratios = [
    '0.5', '1.0', '1.5', '2.0', 
    '2.5', '3.0', '3.5', '4.0'
    ]

for l, pruner in enumerate(pruners):
    for i, ratio in enumerate(tqdm(ratios, desc=pruner)): 
        for r, run_num in enumerate(runs):
            experimentid      = get_experimentid(pruner, ratio, pepochs[pruner], run_num)
            model_file        = experimentid + 'model.pt'
            compression_file  = experimentid + 'compression.pkl'
            compression       = pd.read_pickle(compression_file)
            prunable          = compression['prunable']==True
            weight            = compression['param']=='weight'
            modules           = compression[np.logical_and(prunable, weight)]['module']
            densities         = np.array(compression[np.logical_and(prunable, weight)]['density'])
            shape             = compression[np.logical_and(prunable,weight)]['shape']
            e                 = np.array([np.prod(s) for s in shape]) * (densities)
            with open(experimentid + 'args.json') as f:
                args = json.load(f)
            input_shape, num_classes = load.dimension(args['dataset']) 
            loss = nn.CrossEntropyLoss()
            if i == 0 and r == 0:
                test_loader, _, _ = load.dataloader(
                    args['dataset'], args['test_batch_size'], args['test_batch_size'],
                    False, args['workers'], 0, 0)
                dataset = args['dataset']
                test_batch_size = args['test_batch_size']
                workers = args['workers']
                (data, _) = next(iter(test_loader))
                input_dim = list(data[0,:].shape)
                input = torch.ones([1] + input_dim).to('cuda')

            model = load.model(args['model'], args['model_class'])(
                input_shape, num_classes, args['dense_classifier'], args['pretrained']).to('cuda')
            model.load_state_dict(torch.load(model_file))
            print(f'Forward :')
            model.set_effective_sparsity(is_forward=True, x=input)
            print(f'Backward :')
            model.set_effective_sparsity(is_forward=False, x=None)

            torch.save(model.state_dict(), f"{experimentid + 'model_effective_comp.pt'}")

            effective_densities = []
            set_modules = set(modules.unique())
            for n, m in model.named_modules():
                if hasattr(m, 'weight_mask') and n in set_modules:
                    effective_densities.append((m.weight_mask.sum() / m.weight_mask.numel()).to('cpu'))
            effective_densities = np.array(effective_densities)
            e = np.array([np.prod(s) for s in shape]) * (effective_densities)
            if np.all(e == 0):                
                actual_comp = float('nan')
            else:
                actual_comp = np.sum(np.array([np.prod(s) for s in shape])) / np.sum(e)
            print(np.log10(actual_comp))

In [ ]:
expid = 'cifar100_lottery_vgg16_bn'
ratios = [
    '0.5', '1.0', '1.5', '2.0', 
    '2.5', '3.0', '3.5', '4.0', 
    '4.5', '5.0', '5.5', '6.0', 
    ]

for l, pruner in enumerate(pruners):
    for i, ratio in enumerate(tqdm(ratios, desc=pruner)): 
        for r, run_num in enumerate(runs):
            experimentid      = get_experimentid(pruner, ratio, pepochs[pruner], run_num)
            model_file        = experimentid + 'model.pt'

            with open(experimentid + 'args.json') as f:
                args = json.load(f)
            input_shape, num_classes = load.dimension(args['dataset']) 
            loss = nn.CrossEntropyLoss()
            if i == 0 and r == 0:
                test_loader, _, _ = load.dataloader(
                    args['dataset'], args['test_batch_size'], args['test_batch_size'],
                    False, args['workers'], 0, 0)
                dataset = args['dataset']
                test_batch_size = args['test_batch_size']
                workers = args['workers']
                (data, _) = next(iter(test_loader))
                input_dim = list(data[0,:].shape)
                input = torch.ones([1] + input_dim).to('cuda')

            model = load.model(args['model'], args['model_class'])(
                input_shape, num_classes, args['dense_classifier'], args['pretrained']).to('cuda')
            model.load_state_dict(torch.load(model_file))
            print(f'Forward :')
            model.set_effective_sparsity(is_forward=True, x=input)
            print(f'Backward :')
            model.set_effective_sparsity(is_forward=False, x=None)

            torch.save(model.state_dict(), f"{experimentid + 'model_effective_comp.pt'}")